In [8]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	

    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]


    


####################### Happy Sad ###################
def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]
###########All emotions #####################

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

################ Run Each Method Multiple Times #####################
def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];

##################Main#########################
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for first 30 "+str(HSAccurecy/30))
    print("F1 happy sad for first 30 "+str(HSF1/30))
    print("Accurecy Combined sad for first 30 "+str(CombinedAccurecy/30))
    print("F1 Combined sad for first 30 "+str(CombinedF1/30))
    print("Accurecy Valence for first 30 "+str(ValenceAccurecy/30))
    print("F1 Valence for first 30 "+str(ValenceF1/30))
    print("Accurecy Arousal for first 30 "+str(ArousalAccurecy/30))
    print("F1 Arousal for first 30 "+str(ArousalF1/30))


s01
Array of average Happy Sad accurecies for user s01
Counter({60.0: 46, 40.0: 41, 20.0: 11, 80.0: 2})
Happy Sad
[47.8, 0.36967460317460316]
Array of average All Emotions accurecies for user s01
Counter({25.0: 38, 37.5: 31, 50.0: 12, 12.5: 11, 62.5: 5, 0.0: 3})
Combined Emotions
[31.625, 0.24923962842712843]
Arousal 59.75
Counter({62.5: 34, 50.0: 22, 75.0: 18, 37.5: 15, 87.5: 9, 25.0: 2})
Valence 38.0
Counter({37.5: 46, 50.0: 23, 25.0: 17, 62.5: 7, 12.5: 5, 0.0: 2})
Arousal Valence
[59.75, 38.0, 0.4852532467532468, 0.2826659451659452]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 51, 100.0: 32, 60.0: 14, 40.0: 3})
Happy Sad
[82.4, 0.7525238095238095]
Array of average All Emotions accurecies for user s02
Counter({37.5: 36, 50.0: 31, 62.5: 18, 25.0: 6, 75.0: 6, 12.5: 2, 87.5: 1})
Combined Emotions
[47.375, 0.34330168442668435]
Arousal 62.75
Counter({75.0: 33, 62.5: 28, 50.0: 24, 37.5: 8, 87.5: 6, 25.0: 1})
Valence 70.25
Counter({75.0: 37, 62.5: 25, 87.5: 19, 50.0

Array of average Happy Sad accurecies for user s15
Counter({50.0: 45, 75.0: 31, 25.0: 23, 0.0: 1})
Happy Sad
[51.5, 0.43380952380952387]
Array of average All Emotions accurecies for user s15
Counter({37.5: 36, 25.0: 25, 50.0: 22, 12.5: 10, 0.0: 3, 62.5: 2, 75.0: 2})
Combined Emotions
[34.75, 0.2836706349206349]
Arousal 51.0
Counter({50.0: 29, 62.5: 27, 37.5: 13, 75.0: 13, 25.0: 12, 12.5: 2, 0.0: 2, 87.5: 2})
Valence 59.125
Counter({62.5: 36, 50.0: 32, 75.0: 14, 37.5: 9, 87.5: 5, 100.0: 2, 25.0: 1, 12.5: 1})
Arousal Valence
[51.0, 59.125, 0.45371891996891994, 0.5069453601953602]
s16
Array of average Happy Sad accurecies for user s16
Counter({60.0: 34, 80.0: 31, 40.0: 15, 100.0: 15, 20.0: 4, 0.0: 1})
Happy Sad
[67.0, 0.6309444444444444]
Array of average All Emotions accurecies for user s16
Counter({37.5: 35, 50.0: 26, 25.0: 17, 12.5: 11, 62.5: 6, 0.0: 3, 75.0: 2})
Combined Emotions
[37.0, 0.31368082611832615]
Arousal 48.0
Counter({50.0: 35, 37.5: 27, 62.5: 11, 75.0: 11, 25.0: 8, 12.5: 5,

Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 38, 83.33333333333334: 35, 100.0: 13, 50.0: 12, 16.666666666666664: 2})
Happy Sad
[73.83333333333334, 0.6703258778258777]
Array of average All Emotions accurecies for user s29
Counter({50.0: 37, 75.0: 18, 62.5: 17, 37.5: 16, 25.0: 8, 87.5: 3, 12.5: 1})
Combined Emotions
[53.375, 0.42823945498945504]
Arousal 67.25
Counter({75.0: 34, 50.0: 19, 62.5: 19, 87.5: 14, 37.5: 7, 100.0: 5, 25.0: 2})
Valence 60.875
Counter({50.0: 29, 62.5: 26, 75.0: 25, 37.5: 10, 87.5: 7, 25.0: 2, 100.0: 1})
Arousal Valence
[67.25, 60.875, 0.5870710955710955, 0.5335638250638252]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 34, 57.14285714285714: 28, 85.71428571428571: 26, 42.857142857142854: 10, 100.0: 2})
Happy Sad
[68.85714285714286, 0.5765724275724274]
Array of average All Emotions accurecies for user s30
Counter({62.5: 35, 50.0: 25, 37.5: 18, 75.0: 12, 25.0: 6, 87.5: 4})
Combined Emotions
[55.1